# Generating Unit Cell Data

## Using the ICSD Database

Great database is: http://icsd.cds.rsc.org/search/basic.xhtml   
The search is straightforward: Insert the elements into 'composition'   
Be careful to select experimental references, rather than computational  

One can generate all information regarding a crystal from:
- Space group 
- Lattice constants  
- Lattice vector angles (of the parallelepiped unit cell)  
- Atomic basis of the asymmetric unit (cell)   

## Generating Lattice Vectors

Expressions found from simple geometric arguments. [See the wikipedia reference for more information](https://en.wikipedia.org/wiki/Fractional_coordinates).  
For lattice constants $(a_l,b_l,c_l)$ and angles $(\alpha,\beta,\gamma)$, the lattice vectors $(\mathbf{a},\mathbf{b},\mathbf{c})$ of the **parallelepiped** unit cell in **cartesian** coordinates are given as:  
  
$$\begin{align} 
    &\mathbf{a} = (a_l,0,0)  \\
    &\mathbf{b} = (b_l \cos(\gamma), b_l\sin(\gamma), 0 )        \\
    &\mathbf{c} = (c_l\cos(\beta), c_l \frac{\cos(\alpha)-\cos(\beta)\cos(\gamma)}{\sin(\gamma)}, \frac{\Omega}{a_lb_l\sin(\gamma)}   ) 
  \end{align}  $$  

where the cell volume is, $\Omega=a_l b_l c_l \sqrt{1-\cos(\alpha)^2-\cos(\beta)^2-\cos(\gamma)^2 + 2\cos(\alpha)\cos(\beta)\cos(\gamma)}$



## Generating the Primitive Cell from the Asymmetric Unit

For the space group 152 (for example), there are 6 symmetry operations:

1. $x$, $y$, $z$
2. $\overline{y}$, $x-y$, $\frac{1}{3}+z$  
3. $\overline{x} + y$, $\overline{x}$, $\frac{2}{3} +z$  
4. $y$, $x$, $\overline{z}$
5. $x-y$, $\overline{y}$, $\frac{2}{3} -z$
6. $\overline{x}$, $\overline{x}+y$, $\frac{1}{3} - z$ 

where the overbar imples negative.   

The use of fractional coordinates means that $x$, $y$ and $z$ must be in the limits:  
$0 \leq x \leq 1$    
$0 \leq y \leq 1$   
$0 < z \leq 1/6$ 

If a transformed coordinate lies outside of one of these bounds, then $\pm 1$ should be added as appropriate (or potentially $1\pm x$, for example).

## Converting from Fractional to Cartesian Coordinates 


Fractional coordinates define the basis vectors in the basis/units of the lattice. The Cartesian basis vector $\mathbf{r} = (x,y,z)$ is therefore given as a linear combination of the basis in fractional coordinates multiplied with the lattice vectors:  

$$ \begin{bmatrix}
   x \\
   y \\
   z 
   \end{bmatrix}
= 
 \begin{bmatrix}
  a_{1} & b_{1} & c_{1} \\
  a_{2} & b_{2} & c_{2} \\
  a_{3} & b_{3} & c_{3} 
 \end{bmatrix}
 %
 \begin{bmatrix}
   u \\
   v \\
   w 
   \end{bmatrix}
 =
  \begin{bmatrix}
  a_l & b_l \cos(\gamma) & c_l\cos(\beta) \\
   0  & b_l \sin(\gamma) & c_l \frac{\cos(\alpha)-\cos(\beta)\cos(\gamma)}{\sin(\gamma)} \\
   0  & 0                &\frac{\Omega}{a_lb_l\sin(\gamma)} 
 \end{bmatrix}
  %
 \begin{bmatrix}
   u \\
   v \\
   w 
   \end{bmatrix}
 $$
 $$%Don't think there's a way to express three column vectors as a 3x3 matrix. \mathbf{r} = \mathbf{a}^T \mathbf{b}^T \mathbf{c}^T $$ 


## Generating a Supercell

With knowledge of the lattice vectors and atomic basis, one can generate a super cell according to:  

$$\mathbf{r_{cell}} = n\mathbf{a} + m\mathbf{b} + l\mathbf{c} + \mathbf{r}_{\alpha} $$ 
where $\mathbf{a},\mathbf{b},\mathbf{c}$ are lattice vectors, $(n,m,l)$ are integers, and $\mathbf{r}_{\alpha}=(x,y,z)$ are atomic basis vectors in Cartesian coordinates. 

## $\alpha$-Quartz Example 

The asymmetric cell  in fractional coordinates is:  

|Atom | Basis Vector (frac)          |
|-----|------------------------------|
|Si   |(0.468200, 0.000000, 0.333333)|
|O    |(0.413100, 0.266100, 0.213100)|


One then applies the symmetry operations to each of the basis atoms, resulting in 12 new basis vectors.
In the case of Si, half of the basis vectors are degenerate. Symmetrically equivalent positions can be removed from the basis, leaving a primitive unit cell comprised of 9 basis atoms: 3 Si and 6 O. Note, one **must** apply these operations to the asymmetric basis given in fractional coordinates.   

For example, substituting for the basis vector given for Si into the first three symmetry operations, one obtains:
1. $(x,y,z)$ = $(0.4682, 0.0000, 0.3333)$
2. $(\overline{y}, x-y,\frac{1}{3}+z)$ = $(0., 0.4682, 0.6666)$ 
3. $\overline{x} + y$, $\overline{x}$, $\frac{2}{3} +z$ = $(0.5318, 0.5318, 1.)$   

One finds that symmetry operations 4-6 produce equivalent basis positions after shifting to ensure (x,y,z) are within their defined limits. This is implemented numerically in the python script below. 

In [1]:
#!/usr/bin/env python3

#------------------------------------------------------------------------
# Generate unit cell from asymmetric unit
# - Apply symmetry operations of space group to basis atoms
#   Hard-coded for alpha-quartz
# 
# Refs:
# http://img.chem.ucl.ac.uk/sgp/large/152az1.htm
# http://homepage.univie.ac.at/nikos.pinotsis/spacegroup.html#152
# 
# Note: the condition for if z=0, add 1, which doesn't appear to be the case for x and y
#------------------------------------------------------------------------

#----------------------------------
#Libraries and modules
#----------------------------------
#Libraries 
import sys
import math
import numpy as np


#----------------------------------
# Functions
#----------------------------------

def spacegroup251_operations(i,basis):
    if i==1: pos=operation1(basis[0],basis[1],basis[2])
    if i==2: pos=operation2(basis[0],basis[1],basis[2])
    if i==3: pos=operation3(basis[0],basis[1],basis[2])
    if i==4: pos=operation4(basis[0],basis[1],basis[2])
    if i==5: pos=operation5(basis[0],basis[1],basis[2])
    if i==6: pos=operation6(basis[0],basis[1],basis[2])
    if i<1 or i>6:
        print('Valid symmetry operations 1<i<6. i=',i)
        pos=([0.,0.,0.])
    return pos

def operation1(x,y,z):
    pos = ([x,y,z])
    return pos

def operation2(x,y,z):
    pos = ([-y,x-y,(1./3.)+z])
    return pos

def operation3(x,y,z):
    pos = ([-x+y,-x, (2./3.)+z])
    return pos

def operation4(x,y,z):
    pos = ([y,x,-z])
    return pos

def operation5(x,y,z):
    pos = ([x-y,-y,(2./3.)-z])
    return pos

def operation6(x,y,z):
    pos = ([-x,-x+y,(1./3.)-z])
    return pos

#Fractional coordinates must be in the range [0:1] for (x,y) and [0:1/6] for z
#See, for example:  http://homepage.univie.ac.at/nikos.pinotsis/spacegroup.html#152
def check_fractional_range(basis):
    tol=1.e-5  #Effective zero
    if basis[0] <0.: basis[0]=basis[0]+1.
    if basis[0] >1.: basis[0]=1.-basis[0]
    if basis[1] <0.: basis[1]=basis[1]+1.
    if basis[1] >1.: basis[1]=1.-basis[1]
    if basis[2] <tol: basis[2]=basis[2]+1.
    if basis[2] >(1./.6): basis[2]=1.-basis[2]
    return basis

#----------------------------------
#Main Routine
#----------------------------------
space_group=152
Noperations=6

#Atomic basis in fractional coordinates
asym_basis=np.array( [[0.468200,  0.000000,  0.333333], 
                      [0.413100,  0.266100,  0.213100]])
asym_atom_species=np.array(['Si','O'])

Natoms=Noperations*len(asym_basis)

#Apply to Si and O. 
prim_basis= np.zeros(shape=(Noperations*len(asym_basis),3))

iatom=0
for ib in range(0,len(asym_basis)):
    for i in range(1,Noperations+1):
        prim_basis[iatom,:]=spacegroup251_operations(i,asym_basis[ib,:])
        prim_basis[iatom,:]=check_fractional_range(prim_basis[iatom,:])
        iatom+=1
        

#Round the values to 4 d.p.
prim_basis=np.around(prim_basis, decimals=4)

#Check for duplicate basis atoms and remove
unique_rows,indices = np.unique(prim_basis, axis=0, return_index=True)
                       
#Assign element label for primitive cell atoms retained
atom_type=[None]*len(unique_rows)
for i in range(0,len(unique_rows)):
    #Atom index [1:12]
    atom_index=indices[i]+1
    if float(atom_index)/float(Natoms)<=0.5: atom_type[i]='Si'
    if float(atom_index)/float(Natoms)>0.5: atom_type[i]='O'

#Convert -0 to 0
unique_rows[unique_rows==0.] = 0.
        
#Output result
for i in range(0,len(unique_rows)):
    print(atom_type[i],unique_rows[i,:])

Si [0.     0.4682 0.6667]
O [0.147  0.7339 0.4536]
O [0.2661 0.4131 0.7869]
O [0.4131 0.2661 0.2131]
Si [0.4682 0.     0.3333]
Si [0.5318 0.5318 1.    ]
O [0.5869 0.853  0.1202]
O [0.7339 0.147  0.5464]
O [0.853  0.5869 0.8798]


## Converting $\alpha$-quartz Basis from Fractional to Cartesian

Could combine this with the script above. 
Note, not verified that this is correct. Should use as input in DFTB+ and see if band structure is the same as when fractional coordinates are used for the basis. 

In [1]:
#!/usr/bin/env python3
import sys
import math
import numpy as np

#Set up transformation matrix using period lattice vectors
avec=np.array([ 4.914730, 0.000000, 0.000000])
bvec=np.array([-2.457365, 4.256281, 0.000000])
cvec=np.array([ 0.000000, 0.000000, 5.406570])

T=np.array([avec,bvec,cvec])
T=np.transpose(T)

#Atomic basis in fractional coordinates
basis_f=np.array( [[0.0000, 0.4682, 0.6667], 
                   [0.4682, 0.0000, 0.3333],
                   [0.5318, 0.5318, 1.0000],
                   [0.1470, 0.7339, 0.4536],
                   [0.2661, 0.4131, 0.7869],
                   [0.4131, 0.2661, 0.2131],
                   [0.5869, 0.8530, 0.1202],
                   [0.7339, 0.1470, 0.5464],
                   [0.8530, 0.5869, 0.8798]])
atom_species=np.array(['Si','Si','Si','O','O','O','O','O','O'])

#Atomic basis in Cartesian coordinates (Ang)
basis_ang=np.zeros(shape=(3,len(basis_f)))
basis_ang=np.matmul(T,np.transpose(basis_f))
basis_ang=np.transpose(basis_ang)
print('Cartesian basis positions in Ang:')
for ia in range(0,len(atom_species)):  
    print(atom_species[ia],basis_ang[ia,:])

Cartesian basis positions in Ang, per row:
Si [-1.15053829  1.99279076  3.60456022]
Si [2.30107659 0.         1.80200978]
Si [1.30682671 2.26349024 5.40657   ]
O [-1.08099486  3.12368463  2.45242015]
O [0.29267217 1.75826968 4.25442993]
O [1.37637014 1.13259637 1.15214007]
O [0.78832269 3.63060769 0.64986971]
O [3.24568769 0.62567331 2.95414985]
O [2.75003717 2.49801132 4.75670029]


# Image of Symmetry Operations for Asymmetric Cell of 152 Space Group 
![152_space_operations](Images/152sg.gif)